In [1]:
import time
from datetime import timedelta

import html
import re

import os
import ntpath

import subprocess as sp

import numpy as np
import pandas as pd
import xml.etree.ElementTree as et

import dask
import dask.dataframe as dd
import dask.bag as bd
from dask.distributed import Client
from dask_jobqueue import SLURMCluster

## Read csv files Java Answers

In [2]:
df = dd.read_csv('pmdcodesnippetsviolation_csv/pmdCodeSnippetsViolation*.csv', engine='python', error_bad_lines=False, warn_bad_lines=False, dtype=object)

## Start a Dask cluster using SLURM jobs as workers

In [3]:
#http://jobqueue.dask.org/en/latest/generated/dask_jobqueue.SLURMCluster.html
dask.config.set(
    {
        "distributed.worker.memory.target": False,  # avoid spilling to disk
        "distributed.worker.memory.spill": False,  # avoid spilling to disk
    }
)
cluster = SLURMCluster(
    cores=10, #cores=24, # we set each job to have 1 Worker, each using 10 cores (threads) and 8 GB of memory
    processes=2,
    memory="8GiB",
    walltime="0-00:30",# walltime="0-00:50",
    log_directory="../dask/logs",  # folder for SLURM logs for each worker
    local_directory="../dask",  # folder for workers data
)

/opt/nesi/CS400_centos7_bdw/Python/3.9.5-gimkl-2020a/lib/python3.9/site-packages/distributed/node.py:151: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 45819 instead
  warnings.warn(


Spawn between 20 to 100 workers and connect a client to be able use them.

In [4]:
#cluster.scale(n=20) # ask for 20 jobs or workers
# This also works with adaptive clusters. This automatically launches and kill workers based on load.
# we tell our cluster to autoscale between 10 and 20 workers depending on the load
cluster.adapt(minimum_jobs=10, maximum_jobs=200)
#cluster.adapt(maximum_jobs=20)
client = Client(cluster)
client

Client Scheduler: tcp://192.168.94.163:45063 Dashboard: http://192.168.94.163:45819/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [5]:
df

,beginline,endline,begincolumn,endcolumn,rule,ruleset,class,externalInfoUrl,priority,msg,method
npartitions=1137,,,,,,,,,,,
,object,object,object,object,object,object,object,object,object,object,object
,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...


In [6]:
pmd_codeviolations_init_len = df.shape[0].compute()
print('Initial number of volations {}'.format(pmd_codeviolations_init_len))

Initial number of volations 5653002


In [7]:
# drop empty columns
df = df[~df['class'].isna()] # Drop rows that have NaN in the Code column

In [8]:
pmd_codeviolations_nonan_len =df.shape[0].compute()
print('Number of violations after removing the nans in the class column {}'.format(pmd_codeviolations_nonan_len))

Number of violations after removing the nans in the class column 5642111


In [9]:
print('Difference {}'.format(pmd_codeviolations_init_len - pmd_codeviolations_nonan_len))

Difference 10891


In [10]:
# drop classes that do not have this pattern e.g., Code_10000096_9914015_1959_0
#df = df[df['class'].str.contains('^Code_\d+_\d+_\d+_\d+', regex=True)]
df = df[df['class'].str.contains('^Code_\d+_\d+_\d+_\d+$', regex=True)]

In [11]:
pmd_codeviolations_unkownclass_len = df.shape[0].compute()
print('Number of violations after removing the onces caused by unknown class e.g. inner classes {}'.format(pmd_codeviolations_nonan_len))

Number of violations after removing the onces caused by unknown class e.g. inner classes 5642111


In [12]:
print('Difference {}'.format(pmd_codeviolations_nonan_len - pmd_codeviolations_unkownclass_len))

Difference 585970


#### Count the unique rulesets

In [13]:
#count the unique rulesets
#df.groupby('ruleset').count().shape[0].compute()

#### Get the unique classes dataframe

In [14]:
# get the unique classes
# not the values in the 1st column are where it found the 1st occurances of each unique item
#df['ruleset'].drop_duplicates().compute()

In [15]:
ruleset_df = df[['ruleset']]
#ruleset_df.head()

In [16]:
# This also counts the unique classes
#ruleset_df.index.shape[0].compute()

### Count the number of times each item occured

In [17]:
#https://www.shanelynn.ie/summarising-aggregation-and-grouping-data-in-python-pandas/
ruleset_count_df = pd.DataFrame(ruleset_df.groupby('ruleset')['ruleset'].count().compute())
# rename the column to total
ruleset_count_df.columns=['total']
# drop the index level
ruleset_count_df = ruleset_count_df.reset_index()
ruleset_count_df

,ruleset,total
0,Best Practices,617357
1,Code Style,3016771
2,Design,889718
3,Documentation,20086
4,Error Prone,369698
5,Multithreading,26941
6,Performance,115505
7,Security,65


### Calculate Percentage

In [18]:
ruleset_count_df['percent'] = (ruleset_count_df['total']/ ruleset_count_df['total'].sum())*100

In [23]:
ruleset_count_df['total'].sum()

5056141

In [19]:
ruleset_count_df

,ruleset,total,percent
0,Best Practices,617357,12.210043
1,Code Style,3016771,59.665484
2,Design,889718,17.596780
3,Documentation,20086,0.397259
4,Error Prone,369698,7.311861
5,Multithreading,26941,0.532837
6,Performance,115505,2.284450
7,Security,65,0.001286


### Calculate number to be sampled per ruleset

In [20]:
# https://www.surveysystem.com/sscalc.htm
# put 4887262 into the sample size calculator at 5% confidence interval
#ruleset_count_df['perc384'] = round((ruleset_count_df['percent']/ 100)*384, 0)
#ruleset_count_df

In [21]:
nn

NameError: name 'nn' is not defined

In [ ]:
#### Better computation
import math

def comp_smpl_no(x):
    x = round((x/100)*384)
    if x < 1:
        x = 1
    return x
ruleset_count_df['sample_size_384'] = ruleset_count_df['percent'].apply(comp_smpl_no)
ruleset_count_df

### Group by Ruleset

In [ ]:
grp_df = df.groupby('ruleset')

In [ ]:
grp_dfs = []

In [ ]:
for vio, sample_size in zip(ruleset_count_df['ruleset'], ruleset_count_df['sample_size_384']): # e.g., Best Practices, Coding Style, Design
    # Get all the values belonging to, say Best Practices
    vio_grp = grp_df.get_group(vio)
    vio_grp = vio_grp.reset_index(drop=True).compute()
    #vio_grp.head()
    
    # Randomly select that number
    #https://datatofish.com/random-rows-pandas-dataframe/
    vio_grp_ran = vio_grp.sample(n=sample_size, random_state=10)
    # add the subfolder to get java file
    vio_grp_ran['sample_subfolder'] = subfolder
    # append the data frame to a list
    grp_dfs.append(vio_grp_ran)
    
    ## Make a folder in that directory
    folder = 'sample_pmd'
    # output: path/to/Post.csv => path/to
    mkdir_cmd = 'mkdir {}'.format(folder)
    cmd = sp.run(
        mkdir_cmd, # command
        capture_output=True,
        text=True,
        shell=True
    )
    
    ## Make a folder in that directory
    folder = 'sample_pmd/{}'.format(re.sub(r'\s+', '_', ruleset.lower()))
    # output: path/to/Post.csv => path/to
    mkdir_cmd = 'mkdir {}'.format(folder)
    cmd = sp.run(
        mkdir_cmd, # command
        capture_output=True,
        text=True,
        shell=True
    )
    
    # copy list of sample java files into the destination 
    for filename_to_copy in vio_grp_ran['class']:
        file_to_copy = '{}.java'.format(filename_to_copy)
        copy_file_cmd = 'cp codesnippets_java/{} {}'.format(file_to_copy, folder)
        cmd = sp.run(
            copy_file_cmd, # command
            capture_output=True,
            text=True,
            shell=True
        )
    

### Observed Test

In [ ]:
# Combine all the group dataframe
com_grp_dfs = pd.concat(grp_dfs)
com_grp_dfs.head()

### Sort them in order

In [ ]:
#com_grp_dfs.sort_values('sourcefile', ascending=True, inplace=True)
com_grp_dfs.sort_values(['sample_subfolder','sourcefile'], ascending=[True, True], inplace=True)

In [ ]:
com_grp_dfs.head()

In [ ]:
for subfol, name, ln, msg in zip(com_grp_dfs['sample_subfolder'], com_grp_dfs['sourcefile'], com_grp_dfs['bugline'], com_grp_dfs['shortMsg']):
    print('sample_ckstyl/{}/{} : {} ==> {}'.format(subfol, name, ln, msg))